In [1]:
import sys
sys.path.append('/home/bibek/projects/DEEPL')

In [5]:
from helpers.deep import get_deep_data, get_classifier
#data = get_deep_data(debug=False, filepath='/home/bibek/projects/DEEPL/_playground/sample_data/nlp_out.csv')
import pandas as pd

df = pd.read_csv('/home/bibek/projects/DEEPL/_playground/sample_data/processed_sectors_subsectors.csv')

In [6]:
from helpers.common import (
    rm_punc_not_nums, rm_punc_not_nums_list,
    rm_stop_words_txt, rm_stop_words_txt_list,
    remove_punc_and_nums,
    translate_to_english_txt,
    compose
)
from nltk.stem.porter import PorterStemmer
import random
import langid

from topic_modeling.keywords_extraction import get_key_ngrams

from helpers.functional import curried_map, curried_filter, curried_zip

In [11]:
#################################################
### COMPARE KEYWORDS EXTRACTION VS SIMPLE method
#################################################
# create a function to extract keywords from document(will return list)
def keywords_extracter(doc):
    ngrams = get_key_ngrams(doc, 3)
    allwords = {}
    for x in ngrams['1grams']: 
        allwords[x[0]] = True
    for y in ngrams['2grams']:
        a,b = y[0].split()
        allwords[a] = True
        allwords[b] = True
    for y in ngrams['3grams']:
        a,b,c = y[0].split()
        allwords[a] = True
        allwords[b] = True
        allwords[c] = True
    return list(allwords.keys())

# convert doc to str and then split
str_split = compose(str.split, str)
# stemmer
stemmer = PorterStemmer()
# split and stem
#split_and_stem = lambda x: list(map(stemmer.stem,x))


rm_stop_list = curried_map(rm_stop_words_txt)
rm_punc_list = curried_map(rm_punc_not_nums)
rm_punc_nums_list = curried_map(remove_punc_and_nums)
lower_list = curried_map(str.lower)

composed_list_processor= compose(remove_punc_and_nums, rm_stop_words_txt, str.lower)


# rm_punc_nums_processor
punc_nums_preprocessor = compose(list, curried_filter(lambda x: x.strip()!=''), curried_map(composed_list_processor), str.split, str)

processed = df['excerpt'].apply(punc_nums_preprocessor)
print(processed)
assert False

# keywords pre processor
kw_preprocessor = compose(list, rm_punc_list , rm_stop_list,lower_list,keywords_extracter,str)

# simple pre processor
simple_preprocessor = compose(list, rm_punc_list , rm_stop_list,lower_list,str.split,str)

kw_processed = [(kw_preprocessor(ex), l) for (ex, l) in data]  # if langid.classify(str(ex))[0] == 'en']
simple_processed = [(simple_preprocessor(ex), l) for (ex, l) in data]  # if langid.classify(str(ex))[0] == 'en']
punc_nums_processed = [(punc_nums_preprocessor(ex), l) for (ex, l) in data]  # if langid.classify(str(ex))[0] == 'en']

0          [NN, children, five, risk, acute, malnutrition]
1        [NN, cholera, cases, suspected, NN-NN, october...
2             [NN, people, need, urgent, food, assistance]
3                      [NN, health, facilities, destroyed]
4                         [NN, million, people, need, aid]
5        [humanitarian, needs, said, include, access, s...
6        [people, urgent, food, insecurity, located, ar...
7        [following, government’s, announcement, close,...
8        [exacerbating, pre-existing, displacement, cri...
9        [agencies, expressed, serious, concerns, risk,...
10       [humanitarian, country, team, hct, haiti, coor...
11       [in-kind, contributions, also, made, extend, s...
12       [NN, people, extreme, food, insecurity, affect...
13       [addition, NN, disbursed, emergency, funds, tr...
14       [NN, NN, october, wfp, reached, NN, people, NN...
15       [estimated, NN, people, expected, need, vaccin...
16       [nearly, NN, children, age, five, risk, acute,.

AssertionError: 

In [62]:
from classifier.feature_selectors import UnigramFeatureSelector, BigramFeatureSelector
from classifier.NaiveBayes_classifier import NaiveBayesClassifier

#processed_data = kw_processed
#processed_data = simple_processed

def get_avg_accuracy(iters, size, processed_data, feature_selector=UnigramFeatureSelector):
    sum_accuracy = 0
    accuracies = []
    for x in range(iters):

        random.shuffle(processed_data)
        processed_data = processed_data[:size]

        data_len = len(processed_data)
        test_len = int(data_len * 0.25)

        train_data = processed_data[test_len:]
        test_data = processed_data[:test_len]

        selector = feature_selector.new(corpus=processed_data, top=2000) # use top 2000 words

        classifier = NaiveBayesClassifier.new(selector, train_data)
        
        accuracy = classifier.get_accuracy(test_data)
        accuracies.append(accuracy)
        sum_accuracy += accuracy
    return sum_accuracy/iters, accuracies

In [64]:
SIZE = 200
ITERS = 2

kw_accuracy = get_avg_accuracy(ITERS, SIZE, kw_processed)
punc_nums_accuracy = get_avg_accuracy(ITERS, SIZE, punc_nums_processed)
simple_accuracy = get_avg_accuracy(ITERS, SIZE, simple_processed)
simple_bigram_accuracy = get_avg_accuracy(ITERS, SIZE, simple_processed, BigramFeatureSelector)

print('KEYWORDS:', kw_accuracy)
print('PUNC NUMS:', punc_nums_accuracy)
print('SIMPLE: ', simple_accuracy)
print('SIMPLE BIGRAM: ', simple_bigram_accuracy)

AttributeError: 'list' object has no attribute 'split'